In [12]:
# Import modules
import requests
import pandas as pd

In [38]:
# Create request header
headers = {'User-agent': "jakejoyner9@gmail.com"}

def get_cik_by_index(index, headers=headers):
        # Get all company names/CIK keys
        companyTickers = requests.get(
                "https://www.sec.gov/files/company_tickers.json",
                headers=headers
                )

        # Change from dictionary into dataframe
        companyData = pd.DataFrame.from_dict(companyTickers.json(), orient='index')

        # Add leading zeroes to CIKs
        companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

        # Get the CIK at the inputted index
        cik = companyData[index:index+1].cik_str[0]

        return cik

In [ ]:
def get_submission_data_by_index(index, only_filings_df=False, headers=headers):
        cik = get_cik_by_index(index)
        url = f'https://data.sec.gov/submissions/CIK{cik}.json'
        company_json = requests.get(url, headers=headers).json()
        if only_filings_df:
                return pd.DataFrame(company_json['filings']['recent'])
        return company_json

In [ ]:
data = get_submission_data_by_index(5)

/var/folders/xp/4tbyz70170jbbl56kzhfkjb00000gn/T/ipykernel_10947/2579247315.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cik = companyData[index:index+1].cik_str[0]


In [63]:
data

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000950103-24-016101,2024-11-07,2024-11-05,2024-11-07T18:13:49.000Z,,4,,,,4,4886,0,0,xslF345X05/dp220475_4-newstead.xml,FORM 4
1,0000950103-24-016020,2024-11-05,2024-11-01,2024-11-05T18:26:21.000Z,,4,,,,4,20285,0,0,xslF345X05/dp219135_4-zuckerberg.xml,FORM 4
2,0000950103-24-016019,2024-11-05,2024-11-01,2024-11-05T18:17:09.000Z,,4,,,,4,8397,0,0,xslF345X05/dp220320_4-olivan.xml,FORM 4
3,0001921094-24-001346,2024-11-05,,2024-11-05T16:13:01.000Z,33,144,001-35551,241427610,,144,12197,0,0,xsl144X01/primary_doc.xml,
4,0000950103-24-015834,2024-11-01,2024-10-30,2024-11-01T18:20:20.000Z,,4,,,,4,14748,0,0,xslF345X05/dp220182_4-zuckerberg.xml,FORM 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0000950103-22-002780,2022-02-17,2022-02-15,2022-02-17T19:53:03.000Z,,4,,,,4,24291,0,0,xslF345X03/dp167275_4-cox.xml,FORM 4
996,0000950103-22-002779,2022-02-17,2022-02-15,2022-02-17T19:52:32.000Z,,4,,,,4,7334,0,0,xslF345X03/dp167274_4-houston.xml,FORM 4
997,0000950103-22-002505,2022-02-14,2021-12-31,2022-02-14T18:22:34.000Z,,5,,,,5,8571,0,0,xslF345X03/dp167019_5-schroepfer.xml,FORM 5
998,0000950103-22-002506,2022-02-14,2021-12-31,2022-02-14T18:20:00.000Z,,5,,,,5,11712,0,0,xslF345X03/dp167018_5-cox.xml,FORM 5


In [ ]:
def get_filtered_filings(index, ten_k=True, just_accesion_numbers=False, headers=headers):
    company_filings_df = get_submission_data_by_index(index, only_filings_df=True, headers=headers)
    if ten_k:
        df = company_filings_df[company_filings_df['form'] == '10-K']
    else:
        df = company_filings_df[company_filings_df['form'] == '10-Q']
    if just_accesion_numbers:
        df = df.set_index['reportDate']
        accession_df = df['accessionNumber']
        return accession_df
    else:
        return df